In [1]:
%load_ext autoreload
%autoreload 2

In [102]:
import pandas as pd
import numpy as np
import pickle
import sys
import time
import glob
import re
from tqdm.notebook import trange, tqdm
import classifier_utils as cu
import utils
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [27]:
def load_mention_tweets(username):
    with open('../data/target_users/' + username + '.p', 'rb') as file:
        tweets = pickle.load(file)
    tweets = [t for t in tweets if not hasattr(t, 'retweeted_status')]
    return tweets

In [92]:

model_files = [
#     'final_model_toxic_comments_identity_hate.p',
#     'final_model_toxic_comments_insult.p',
#     'final_model_toxic_comments_obscene.p',
#     'final_model_toxic_comments_severe_toxic.p',
#     'final_model_toxic_comments_threat.p',
#     'final_model_toxic_comments_toxic.p'
    
#     'final_model_gboost_toxic_comments_identity_hate.p',
#     'final_model_gboost_toxic_comments_insult.p',
#     'final_model_gboost_toxic_comments_obscene.p',
#     'final_model_gboost_toxic_comments_severe_toxic.p',
#     'final_model_gboost_toxic_comments_threat.p',
#     'final_model_gboost_toxic_comments_toxic.p',
    
    
#     'final_model_gboost_hate_offensive_Hate.p',
#     'final_model_gboost_hate_offensive_Offensive.p'
        'final_model_gboost_hate_abuse_abusive.p',
        'final_model_gboost_hate_abuse_hateful.p'
]
models = []
for file in model_files:
    with open(file, 'rb') as f:
        name = file.replace('.p', '').replace('final_model_toxic_comments_', '')
        models.append((name, pickle.load(f)))


In [28]:
user_mention_tweets = {}
for i, r in tqdm(list(target_users_df.iterrows())):
    user_mention_tweets[i] = load_mention_tweets(i)

In [63]:
target_users_df = pd.read_csv('../data/target_users.csv', index_col=0)

In [64]:
target_users_df.columns

Index(['gender', 'primary_association', 'category', 'follower_count',
       'tweets_count', 'created_date', 'verified', 'mentions_saved',
       'harassment_tweets_prob', 'harassment_fraction_identity_hate',
       'harassment_fraction_insult', 'harassment_fraction_obscene',
       'harassment_fraction_severe_toxic', 'harassment_fraction_threat',
       'harassment_fraction_toxic', 'harassing_users_identity_hate',
       'harassing_users_insult', 'harassing_users_obscene',
       'harassing_users_severe_toxic', 'harassing_users_threat',
       'harassing_users_toxic',
       'harassment_fraction_final_model_gboost_toxic_comments_identity_hate',
       'harassing_users_final_model_gboost_toxic_comments_identity_hate',
       'harassment_fraction_final_model_gboost_toxic_comments_insult',
       'harassing_users_final_model_gboost_toxic_comments_insult',
       'harassment_fraction_final_model_gboost_toxic_comments_obscene',
       'harassing_users_final_model_gboost_toxic_comments_obsc

In [58]:
target_users_df = target_users_df.loc[np.logical_and(target_users_df['follower_count'] > 10000, target_users_df['follower_count'] < 1e6)]

In [93]:
def compute_harassment_data(model, target_users_df, user_mention_tweets):
    fraction = []
    harassing_users = []
    for i, r in tqdm(list(target_users_df.iterrows())):
        texts = np.array([t.text for t in user_mention_tweets[i]])
        users = np.array([t.user.screen_name for t in user_mention_tweets[i]])
        if len(texts) == 0:
            fraction.append(0)
            harassing_users.append([])
            continue
        probs = model.predict_proba(texts)
        p = probs.mean(axis=0)[1]
        fraction.append(p)
        harassing_users.append([u for u in users[probs[:, 1] > 0.6]])
        
    return fraction, harassing_users


In [94]:
for model_name, model in models:
    f, u = compute_harassment_data(model, target_users_df, user_mention_tweets)
    target_users_df['harassment_fraction_' + model_name] = f
    target_users_df['harassing_users_' + model_name] = u

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'becau'] not in stop_words.



In [100]:
target_users_df.columns

Index(['gender', 'primary_association', 'category', 'follower_count',
       'tweets_count', 'created_date', 'verified', 'mentions_saved',
       'harassment_tweets_prob', 'harassment_fraction_identity_hate',
       'harassment_fraction_insult', 'harassment_fraction_obscene',
       'harassment_fraction_severe_toxic', 'harassment_fraction_threat',
       'harassment_fraction_toxic', 'harassing_users_identity_hate',
       'harassing_users_insult', 'harassing_users_obscene',
       'harassing_users_severe_toxic', 'harassing_users_threat',
       'harassing_users_toxic',
       'harassment_fraction_final_model_gboost_toxic_comments_identity_hate',
       'harassing_users_final_model_gboost_toxic_comments_identity_hate',
       'harassment_fraction_final_model_gboost_toxic_comments_insult',
       'harassing_users_final_model_gboost_toxic_comments_insult',
       'harassment_fraction_final_model_gboost_toxic_comments_obscene',
       'harassing_users_final_model_gboost_toxic_comments_obsc

In [143]:
frac_columns = [
    'harassment_fraction_final_model_gboost_toxic_comments_toxic',
#     'harassment_fraction_final_model_gboost_toxic_comments_severe_toxic',
    'harassment_fraction_final_model_gboost_toxic_comments_obscene',
#     'harassment_fraction_final_model_gboost_toxic_comments_threat',
    'harassment_fraction_final_model_gboost_toxic_comments_insult',
#     'harassment_fraction_final_model_gboost_toxic_comments_identity_hate',
       
    'harassment_fraction_final_model_gboost_hate_abuse_abusive',
    'harassment_fraction_final_model_gboost_hate_abuse_hateful', 
       
    'harassment_fraction_final_model_gboost_hate_offensive_Offensive',
    'harassment_fraction_final_model_gboost_hate_offensive_Hate',
    
]

names = [
    '$D_1 \\text{: Toxic}$',
#     '$D_1 \\text{: Severe Toxic}$',
    '$D_1 \\text{: Obscene}$',
#     '$D_1 \\text{: Threat}$',
    '$D_1 \\text{: Insult}$',
#     '$D_1 \\text{: Identity Hate}$',
    
    
    '$D_2 \\text{: Abusive}$',
    '$D_2 \\text{: Hateful}$',
    
    '$D_3 \\text{: Offensive}$',
    '$D_3 \\text{: Hate}$',
    
]


In [162]:

fig = make_subplots(rows=4, cols=2, subplot_titles=names)
fig.update_layout(
    autosize=False,
    width=500,
    height=500)

fig.update_layout(
    legend=dict(x=0.6, y=0),
    margin=dict(l=20, r=20, t=20, b=20)
)

df = target_users_df.loc[target_users_df['gender']=='female']
c = 0
showlegend=True
for col_name in frac_columns:
    all_scores = df[col_name]
    p_scores = df.loc[df['category']=='politician'][col_name]
    j_scores = df.loc[df['category']=='journalist'][col_name]
    bin_edges = np.histogram_bin_edges(all_scores, bins='auto', range=(0.0, 0.4))
    
    weights = np.ones_like(p_scores)/(len(p_scores))
    hist, _= np.histogram(p_scores, bins=bin_edges, weights=weights)
    
    fig.add_trace(
        go.Scatter(
            x=bin_edges, y=hist, 
            name='Politician', 
            mode='lines', 
            legendgroup="group", 
            showlegend=showlegend,
            line=dict(color="#1f77b4"),
        ),
        col= c % 2 + 1, row= c // 2 + 1
    )
    
    weights = np.ones_like(j_scores)/(len(j_scores))
    hist, _= np.histogram(j_scores, bins=bin_edges, weights=weights)
    fig.add_trace(
        go.Scatter(
            x=bin_edges, y=hist,
            name='Journalist', 
            mode='lines', 
            legendgroup="group", 
            showlegend=showlegend,
            line=dict(color="#ff7f0e"),
        ),
        col= c % 2 + 1, row= c // 2 + 1
    )
    c += 1
    showlegend = False
# fig.write_image("figures/victim_harassment_fractions.png")
fig.show()


In [ ]:
target_users_df

In [99]:

fig = go.Figure()
fig.add_trace(go.Histogram(name='Female', x=target_users_df.loc[np.logical_and(target_users_df['category']=='politician', target_users_df['gender']=='female')]['harassment_fraction_final_model_gboost_hate_abuse_abusive']))
fig.add_trace(go.Histogram(name='Male', x=target_users_df.loc[np.logical_and(target_users_df['category']=='politician', target_users_df['gender']=='male')]['harassment_fraction_final_model_gboost_hate_abuse_abusive']))
fig.show()

In [33]:
user_mention_tweets['SenTinaSmith'][0].user.screen_name

'mamalama317'

KeyboardInterrupt: 

In [24]:
target_users_df['harassment_tweets_prob'] = d

In [23]:
d = []
for i, r in tqdm(target_users_df.iterrows()):
    tweets = load_mention_tweets(i)
    if len(tweets) == 0:
        d.append(0)
        continue
    probs = model.predict_proba(tweets)
    p = probs.mean(axis=0)[1]
    d.append(p)
    print(i, p)

SenTinaSmith 0.29216006216006224
gillibrandny 0.28347305389221544
SenJackyRosen 0.30182696259761616
SenCortezMasto 0.2996283185840707
SenKamalaHarris 0.3482513368983956
SenMcSallyAZ 0.2847387387387387
SenJoniErnst 0.3124612811506945
SenatorSinema 0.3106013363028956
SenatorBaldwin 0.32
SenatorFischer 0.28315177304964495
SenDuckworth 0.29849390243902446
SenatorHassan 0.30122874806800626
PattyMurray 0.3173519579751668
MarshaBlackburn 0.325810479375697
SenCapito 0.3736275565123751
SenatorCantwell 0.2711053240740736
SenatorShaheen 0.3055358045492844
maziehirono 0.33020356394129924
SenStabenow 0.29807613412228795
SenAmyKlobuchar 0.31826070175438553
SenatorCollins 0.3592289719626176
lisamurkowski 0.31367093360641773
SenHawleyPress 0.2844927536231887
SenatorRomney 0.31683486735246513
SenBillCassidy 0.3069753086419755
SenDougJones 0.29161691542288576
SenJohnKennedy 0.3194756097561001
SenThomTillis 0.32797780881440086
SenSasse 0.31031460437710506
SenatorRounds 0.31535200311163064
SenDanSullivan 

In [97]:
target_users_df.to_csv('../data/target_users.csv')

In [62]:
target_users_df.sort_values('harassment_tweets_prob', ascending=False).head(50)

,gender,primary_association,category,follower_count,tweets_count,created_date,verified,mentions_saved,harassment_tweets_prob,harassment_fraction_identity_hate,...,harassment_fraction_final_model_gboost_toxic_comments_insult,harassing_users_final_model_gboost_toxic_comments_insult,harassment_fraction_final_model_gboost_toxic_comments_obscene,harassing_users_final_model_gboost_toxic_comments_obscene,harassment_fraction_final_model_gboost_toxic_comments_severe_toxic,harassing_users_final_model_gboost_toxic_comments_severe_toxic,harassment_fraction_final_model_gboost_toxic_comments_threat,harassing_users_final_model_gboost_toxic_comments_threat,harassment_fraction_final_model_gboost_toxic_comments_toxic,harassing_users_final_model_gboost_toxic_comments_toxic
username,,,,,,,,,,,,,,,,,,,,,
katierogers,female,nyt,journalist,57905,66,2008-03-10 19:58:53,True,True,0.442179,0.012629,...,0.073074,"[pressp_photo, RiskyLiberal, Vetipie, Coughily...",0.067223,"[MissingLynx, Vetipie, 2emesouffle, benyankee,...",0.006981,[],0.002240,[],0.155706,"[pressp_photo, STrimel, RiskyLiberal, JulesvSm..."
samcunningham,male,dailymail,journalist,13277,13761,2009-02-17 15:38:08,True,True,0.441409,0.011297,...,0.040386,[],0.041009,"[Giggsy57, mcfcpipe, mikeroyle7]",0.004638,[],0.001349,[],0.106158,"[Giggsy57, mcfcpipe, maidagfj, mikeroyle7]"
samcunningham,male,nyt,journalist,13277,13761,2009-02-17 15:38:08,True,True,0.441409,0.011297,...,0.040386,[],0.041009,"[Giggsy57, mcfcpipe, mikeroyle7]",0.004638,[],0.001349,[],0.106158,"[Giggsy57, mcfcpipe, maidagfj, mikeroyle7]"
lauriegnyt,female,nyt,journalist,18546,8101,2012-01-10 18:40:28,True,True,0.422500,0.018750,...,0.016971,[],0.011849,[],0.001677,[],0.001038,[],0.048664,[]
jennydeluxe,female,nyt,journalist,705522,34002,2007-11-21 23:47:58,True,True,0.403485,0.007879,...,0.022763,[heyteebs],0.022082,[heyteebs],0.003932,[],0.001138,[],0.057787,[heyteebs]
brandondarby,male,britebart,journalist,72108,69244,2011-02-01 04:46:09,True,True,0.402099,0.012048,...,0.040070,"[9449sarlo, bryson80451340, tsar_tribunal, Tim...",0.038831,"[9449sarlo, ShareTheTruth4, chase94584356, tsa...",0.004137,[],0.001330,[],0.106088,"[9449sarlo, ShareTheTruth4, chase94584356, bry..."
BazBam,male,nyt,journalist,20614,15761,2009-03-16 13:02:02,True,True,0.399919,0.012146,...,0.015701,[],0.016084,[hughspeaight],0.001942,[],0.003603,[],0.055089,[hughspeaight]
BazBam,male,dailymail,journalist,20614,15761,2009-03-16 13:02:02,True,True,0.399919,0.012146,...,0.015701,[],0.016084,[hughspeaight],0.001942,[],0.003603,[],0.055089,[hughspeaight]
FoyChris,male,nyt,journalist,11102,7738,2011-09-10 04:33:04,True,True,0.396667,0.003333,...,0.019873,[],0.018460,[],0.002389,[],0.001190,[],0.076838,[]


In [30]:
for i, r in tqdm(target_users_df.iterrows()):
    tweets = load_mention_tweets(i)
    probs = model.predict_proba(tweets)[:, 1]
    h = np.array(tweets)[probs > 0.9]
    if len(h) > 0:
        print(h)

['@RedWalrus1 @gillibrandny Who the hell paid for that farce']
['@SenKamalaHarris karma....it’s a bitch.']
['@SenCapito Crooked ass']
['@CNNSotu @amyklobuchar @SenAmyKlobuchar Let me guess more Trump Bull Shit']
['@SenThomTillis @realDonaldTrump You’re the worst']


KeyboardInterrupt: 